In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
session = requests.Session()

params = {'username': "k33shen", 'password': "", '_eventId': 'submit', 'submit':'LOGIN', 'lt': 'e1s1'}

In [4]:
def find_total_page_count(bsObj):
    """Returns the total number of postings"""
    posting_count = bsObj.find("div", {"id": "postingsTablePlaceholder"}).find("span", {"class": "badge badge-info"}).get_text().strip()
    return (int(posting_count) - 1) // 100 + 1 #because math module isn't found so math.ceil is unavailable

def find_action_value(bsObj):
    script = bsObj.find('div',{'class': 'tab-content'}).find('script')
    script = str(script)
    function_start_index = script.find('function loadPostingTable')
    first_apostrophe_index = script.find("'", function_start_index)
    second_apostrophe_index = script.find("'", first_apostrophe_index + 1)
    return script[first_apostrophe_index + 1:second_apostrophe_index]

In [5]:
with requests.Session() as s:

    #connection = create_database()
    s.get('https://cas.uwaterloo.ca/cas/login?service=https://waterlooworks.uwaterloo.ca/waterloo.htm')
    p = s.post('https://cas.uwaterloo.ca/cas/login?service=https://waterlooworks.uwaterloo.ca/waterloo.htm', data=params)
    print('==================')

    #go to all postings

    postings_home_page = s.get('https://waterlooworks.uwaterloo.ca/myAccount/hire-waterloo/full-time-jobs/jobs-postings.htm')

    #get the value for 'Content-Disposition: form-data; name="action"'
    bsObj = BeautifulSoup(postings_home_page._content, "html.parser")
    a = bsObj.find("form", {"id": "widgetSearch"})
    print(a)
    action_value = bsObj.find("form", {"id": "widgetSearch"}).find("input", {"name": "action"})['value']
    initial_search_action_value = bsObj.find("form", {"id": "widgetSearch"}).find("input", {"name": "initialSearchAction"})['value']
    
    #search data is gotten via the html info from Inspect elements
    search_data = {
    	'action': action_value,
        'initialSearchAction' : initial_search_action_value,
    	'filter': 'all',
    	'widget1Keyword': '',
    	'widget2Keyword': ''
    }

    files = {'': ('', '')}


    search_page = s.post('https://waterlooworks.uwaterloo.ca/myAccount/hire-waterloo/full-time-jobs/jobs-postings.htm', data=search_data, files=files)

    #####look for the action value in order to go to next page. It's inside
    '''
    function
    loadPostingTable(orderBy, access, customItem, callback, page)
    {
        var
    request = {
        action: '_-_-DtDt7fikcywBvXJ5_GeUVj5FoLjMzUnTyXWgiil8BBQ8Byq4neMkVyIpv2ETW2g6BMkCoBSAjurw07r5KrzL8hKieUeLQOeVo6IIf31ccKMHDBISGKNPo3--CDlArYy0RyzZlG_lKqUNlJAVNNiiWA6pqu1mFPyKBMbqXXSBP_st',
        orderBy: orderBy,
        postings: access,
        page: page,
        initialSearchAction: '',
        searchType: '',
        npfGroup: $('[name=npfGroup]').val(),
                   rand: Math.floor(Math.random() * 100000)
    };
    if (customItem)
    '''
    bsObj = BeautifulSoup(search_page._content, "html.parser")

    total_page_count = find_total_page_count(bsObj)
    action_value_100_postings = find_action_value(bsObj)
    
    rows = []
    for page_count in range(1, total_page_count + 1): #stop at total_page_count + 1 because range is not inclusive
        print ('========================= page_count: ' + str(page_count) + '==========================================')
        data_100_postings = {
            'action': action_value_100_postings,
            'initialSearchAction' : initial_search_action_value,
            'sortDirection': 'Forward',
            'postings': 'infoForPostings',
            'page': str(page_count)
        }

        page = s.post('https://waterlooworks.uwaterloo.ca/myAccount/hire-waterloo/full-time-jobs/jobs-postings.htm', data=data_100_postings)
        bsObj = BeautifulSoup(page._content, "html.parser")
        tbody = ""
        try:
            tbody = bsObj.select("table#postingsTable > tbody")[0]
        except:
            print('=============== OH NO!! =================')
            print(page.text)

        postings = tbody.find_all('tr')
       
        #### Look at all jobs on page
        for i in range(len(postings)):

            tds = postings[i].find_all('td')
            id = tds[2].get_text() # used to be tds[1], now it's tds[2]. There are changes every once in a while.
            print(id)

            jsFunction = tds[3].find('a')['onclick'] # used to be tds[2], now tds[3]
            print(jsFunction)

            # This is hard-coded and brittle but I want a quick fix
            # orbisApp.buildForm({'action':'THISISIT!',....})

            startIndex = 30
            endIndex = jsFunction.find("'", startIndex + 1)

            action_value = jsFunction[startIndex : endIndex]
            print(action_value)

            '''
            #get initialSearchAction
            startIndex = endIndex + 25
            endIndex = jsFunction.find("'", startIndex+1)
            initial_search_action = jsFunction[startIndex : endIndex]
            print(initial_search_action)
            '''


            '''
            path = tds[2].find('a')['href']
            print(path)
            intermediate_page = s.get('https://waterlooworks.uwaterloo.ca/myAccount/hire-waterloo/full-time-jobs/jobs-postings.htm' + path) #to get action to go to real page
            print (intermediate_page)
            bsObj = BeautifulSoup(intermediate_page._content, "html.parser")
            print(bsObj)
            action_value = bsObj.find("form", {"id": "rform"}).find("input", {"name": "action"})['value']
            '''

            data = {
                'action': action_value,
                'initialSearchAction': initial_search_action_value,
                'searchType':'',
                'accessToPostings':'infoForPostings',
                'postingId': id,
                'npfGroup':'',
                'sortDirection': 'Reverse',
            }


            files = {'': ('', '')}
            posting_page = s.post('https://waterlooworks.uwaterloo.ca/myAccount/hire-waterloo/full-time-jobs/jobs-postings.htm', data=data, files=files)
            bsObj = BeautifulSoup(posting_page._content, "html.parser")

            #finding company info, the top <h5> for some reason isn't being found. Maybe because of the ::before
            organization_name = ""
            division_name = ""

            try:
                org_table = bsObj.find_all('table')[-1]
                organization_name = (org_table.find_all('tr')[0]).find_all('td')[1].get_text().strip()
                division_name = (org_table.find_all('tr')[1]).find_all('td')[1].get_text().strip()
                print(organization_name)
                
                app_delivery_table = bsObj.find_all('table')[-2]
                
                trs = bsObj.find('div', {'id': 'postingDiv'}).find('table').find('tbody').find_all('tr', recursive=False)
                app_table_trs = app_delivery_table.find('tbody').find_all('tr', recursive=False)

#                 info = {
# 	                'Organization:': organization_name,
# 	                'Division:': division_name,
# 	                'Work Term:' : '',
# 	                'Job Type:' : '',
# 	                'Job Title:' : '',
# 	                'Number of Job Openings:':'',
# 	                'Job Category (NOC):':'',
# 	                'Level:':'',
# 	                'Region:':'',
# 	                'Job - Address Line One:':'',
# 	                'Job - Address Line Two:':'',
# 	                'Job - City:':'',
# 	                'Job - Province / State:': '',
# 	                'Job - Postal Code / Zip Code (X#X #X#):': '',
# 	                'Job - Country:': '',
# 	                'Work Term Duration:': '',
# 	                'Job Summary:': '',
# 	                'Job Responsibilities:': '',
# 	                'Required Skills:': '',
# 	                'Targeted Degrees and Disciplines:':'',
# 	                'Transportation and Housing:': '',
# 	                'Compensation and Benefits Information:':'',
# 	                'Additional Information:': '',
# 	                'Additional Job Identifiers:':'',
# 	                'Special Job Requirements:':'',
# 	                'Other':''
# 	            }
                
                #FUll time job structure
                info = {
	                'Organization:': organization_name,
	                'Division:': division_name,
	                'Term Posted:' : '',
	                'Position Type:': '',
	                'Job Title:' : '',
	                'Job Openings:':'',
	                'Job Category (NOC):':'',
	                'Region:':'',
	                'Start Date:': '',
	                'Job Summary:': '',
	                'Job Responsibilities:': '',
	                'Required Skills:': '',
	                'Targeted Degrees and Disciplines:': '',
                    'Transportation and Housing:': '',
	                'Compensation and Benefits Information:':'',
	                'Additional Information:': '',
	                'Additional Job Identifiers:':'',
	                'Special Job Requirements:':'',
                    'Application Deadline:': '',
                    'Application Delivery:': '',
                    'If by eMail, send to:': '',
                    'Application Documents Required:':'',
	                'Other':''
	            }


                for tr in trs:
                    tds = tr.find_all('td')
                    type = tds[0].get_text().strip()
                    value = tds[1].get_text().strip()
                    if type in info:
                        info[type] = value
                    else:
                        info['Other'] = value
                        
                for tr in app_table_trs:
                    tds = tr.find_all('td')
                    type = tds[0].get_text().strip()
                    value = tds[1].get_text().strip()
                    if type in info:
                        info[type] = value
                    else:
                        info['Other'] = value                   

                print("COLLECTED INFO")
                #print(info)
                
                #append to rows array
                rows.append(info)
                
                
            except:
                print("Something bad happened and this job was not stored")


<form enctype="multipart/form-data" id="widgetSearch" method="post">
<input name="action" type="hidden" value="_-_-tLxfOwoIxieB-7rMJ1cmyejJUxcNEIEpJTk-fEc5VfJPSETESp-9hxHuhco5yXU4JCz6w_UbROmx-FE5Zsg4uLqSwq8UNwUjb_I6uVg8OlbKMbpoiDz55V8LTfST-J1jaG_3EZiNSTt7RhTuCsemNh-qQGbTaverDbMzwsc"/>
<input name="filter" type="hidden" value="all"/>
<input name="initialSearchAction" type="hidden" value="_-_-JYdEesI833o0kTS57aua7P9FC8vmqM7sWFvVO5_sP-wqeEd2FH-5-m64QNFyKLnfc1dmmrhKrc5DQRlzR523fwrklPw_UMyOsmCRfjY2jgHl4C2g8CLHarqbuo4FsvPlleJNy8IIZaDIOdZdfmRS99XhNmVgESX1POMKM5TnRJol72o"/>
<div class="spaceForSearchFilters">
<div class="display--flex form--col--3">
<div class="fqDiv margin--r--m">
<label>
										Keyword<br/>
<input name="widget1Keyword" type="text">
</input></label>
</div>
<div class="fqDiv margin--r--m">
<label>Region:
											<br/>
<input name="filter2" type="hidden" value="exact">
<select id="widget2Keyword" multiple="multiple" name="widget2Keyword" size="5" style="width: inherit;">


In [6]:
#save to pandas dataframe
df = pd.DataFrame.from_dict(rows, orient="columns")

In [7]:
len(rows)

237

In [8]:
pd.set_option('display.max_columns', None)  
df.head()

,Additional Information:,Additional Job Identifiers:,Application Deadline:,Application Delivery:,Application Documents Required:,Compensation and Benefits Information:,Division:,"If by eMail, send to:",Job Category (NOC):,Job Openings:,Job Responsibilities:,Job Summary:,Job Title:,Organization:,Other,Position Type:,Region:,Required Skills:,Special Job Requirements:,Start Date:,Targeted Degrees and Disciplines:,Term Posted:,Transportation and Housing:
0,,,"Aug 21, 2019 11:59:00 PM",Apply direct to company website,Résumé,,Divisional Office,,2173 Software Engineers and Designers,1,"What You'll Do\nAt the core of this role, you ...",1QBit is dedicated to solving the world's most...,SOFTWARE DEVELOPER,1QB Information Technologies Inc,Please apply using the following link: https:/...,Full-time,Western Canada,What You'll Bring\r\nMembers of our team bring...,,08/12/2019,Targeted Clusters\r\n\t\t\t\n\r\n\t\t\r\n\t\t\...,2019 July to 2020 June,
1,Unrestricted Driver Licence,,"Aug 27, 2019 11:59:00 PM",Apply direct to company website,Résumé,,Divisional Office,,2173 Software Engineers and Designers,6,General Job Duties:\r\nEvaluating existing dat...,"As a top technology firm, 3-GIS helps the worl...",Solutions Engineer,3-GIS,https://3gis.applicantpro.com/jobs/,Full-time,USA - South,Experience & Expectations:\r\nExceptional comm...,,,Targeted Clusters\r\n\t\t\t\n\r\n\t\t\r\n\t\t\...,2019 July to 2020 June,Relocation packages available.
2,,,"Aug 27, 2019 11:59:00 PM",Apply direct to company website,Résumé,,Divisional Office,,"2148 Other Professional Engineers, n.e.c",3,Responsibilities: \r\nPerform and maintain sof...,"As a top technology firm, 3-GIS helps the worl...",GIS Testing Analyst,3-GIS,https://3gis.applicantpro.com/jobs/,Full-time,USA - South,Required Qualifications:\r\nB.S. in computer s...,,,Targeted Clusters\r\n\t\t\t\n\r\n\t\t\r\n\t\t\...,2019 July to 2020 June,Relocation package offered.
3,,,"Jul 17, 2020 09:00:00 AM",Apply direct via email,"Cover Letter,Résumé",,Divisional Office,careers@7dsurgical.com,2173 Software Engineers and Designers,1,"Responsibilities\n\nDesign, develop, and unit ...",At 7D Surgical you will find a diverse and vib...,Software Developer (Junior),7D Surgical,What Makes Us Attractive?\n\nOur mighty team o...,Graduating,ON - Toronto,Qualifications\n\nBachelor's or graduate degre...,,,Targeted Clusters\r\n\t\t\t\n\r\n\t\t\r\n\t\t\...,2019 July to 2020 June,
4,,,"Aug 26, 2019 09:00:00 AM",Apply direct via email,"Cover Letter,Résumé",,Divisional Office,careers@adastragrp.com,2173 Software Engineers and Designers,2,Work with a team of Adastra consultants in the...,Adastra's Nova Consultants are new and recent ...,Nova Consultant - Data Management,Adastra Corporation,Please apply by submitting your resume with a ...,Full-time,ON - Toronto,"Computer Science, Computer Engineering, or sim...",,09/03/2019,Targeted Clusters\r\n\t\t\t\n\r\n\t\t\r\n\t\t\...,2019 July to 2020 June,


In [9]:
print(df.columns)

Index(['Additional Information:', 'Additional Job Identifiers:',
       'Application Deadline:', 'Application Delivery:',
       'Application Documents Required:',
       'Compensation and Benefits Information:', 'Division:',
       'If by eMail, send to:', 'Job Category (NOC):', 'Job Openings:',
       'Job Responsibilities:', 'Job Summary:', 'Job Title:', 'Organization:',
       'Other', 'Position Type:', 'Region:', 'Required Skills:',
       'Special Job Requirements:', 'Start Date:',
       'Targeted Degrees and Disciplines:', 'Term Posted:',
       'Transportation and Housing:'],
      dtype='object')


In [10]:
df.to_hdf('data.h5', key="s")

In [11]:
test = pd.read_hdf('data.h5', 's')

In [12]:
len(test)

237